In [2]:
HOUSING_PATH = "data/housing.csv"

In [5]:
import pandas as pd
import numpy as np
np.random.seed(42)

#housing = pd.read_csv(HOUSING_PATH)
#housing.head()

In [8]:
'''longitude 	latitude	housing_median_age 	total_rooms 	total_bedrooms 	population 	households 	median_income 	median_house_value 	ocean_proximity
0 	-122.23 	37.88 	41.0 	880.0 	129.0 	322.0 	126.0 	8.3252 	452600.0 	NEAR BAY
1 	-122.22 	37.86 	21.0 	7099.0 	1106.0 	2401.0 	1138.0 	8.3014 	358500.0 	NEAR BAY
2 	-122.24 	37.85 	52.0 	1467.0 	190.0 	496.0 	177.0 	7.2574 	352100.0 	NEAR BAY
3 	-122.25 	37.85 	52.0 	1274.0 	235.0 	558.0 	219.0 	5.6431 	341300.0 	NEAR BAY
4 	-122.25 	37.85 	52.0 	1627.0 	280.0 	565.0 	259.0 	3.8462 	342200.0 	NEAR BAY'''

'longitude \tlatitude\thousing_median_age \ttotal_rooms \ttotal_bedrooms \tpopulation \thouseholds \tmedian_income \tmedian_house_value \tocean_proximity\n0 \t-122.23 \t37.88 \t41.0 \t880.0 \t129.0 \t322.0 \t126.0 \t8.3252 \t452600.0 \tNEAR BAY\n1 \t-122.22 \t37.86 \t21.0 \t7099.0 \t1106.0 \t2401.0 \t1138.0 \t8.3014 \t358500.0 \tNEAR BAY\n2 \t-122.24 \t37.85 \t52.0 \t1467.0 \t190.0 \t496.0 \t177.0 \t7.2574 \t352100.0 \tNEAR BAY\n3 \t-122.25 \t37.85 \t52.0 \t1274.0 \t235.0 \t558.0 \t219.0 \t5.6431 \t341300.0 \tNEAR BAY\n4 \t-122.25 \t37.85 \t52.0 \t1627.0 \t280.0 \t565.0 \t259.0 \t3.8462 \t342200.0 \tNEAR BAY'

In [ ]:
housing.info()

In [ ]:
# Categ. features
housing["ocean_proximity"].value_counts()

In [ ]:
housing.describe()

In [ ]:
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20,15))

# Training & test

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

In [ ]:
housing["median_income"].hist()

In [ ]:
# Divide by 1.5 to limit the number of income categories
housing["income_cat"] = np.ceil(housing["median_income"] / 1.5)
# Label those above 5 as 5
housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True)
housing["income_cat"].hist()

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

# Visualization

In [ ]:
housing = strat_train_set.copy()

_ = housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1)

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,
    s=housing["population"]/100, label="population", figsize=(10,7),
    c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True,
    sharex=False)
_ = plt.legend()

In [ ]:
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
from pandas.plotting import scatter_matrix

attributes = ["median_house_value", "median_income", "total_rooms", "housing_median_age"]
_ = scatter_matrix(housing[attributes], figsize=(15, 10))

In [ ]:
housing.plot(kind="scatter", x="median_income", y="median_house_value", alpha=0.1)
plt.axis([0, 16, 0, 550000])

In [ ]:
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"]=housing["population"]/housing["households"]


In [ ]:
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

# Data preprocessing

In [1]:
housing = strat_train_set.drop("median_house_value", axis=1) # X
housing_labels = strat_train_set["median_house_value"].copy() # Y

NameError: name 'strat_train_set' is not defined

In [ ]:
# вспоминаем, что у нас есть null значения
sample_incomplete_rows = housing[housing.isnull().any(axis=1)].head()
sample_incomplete_rows

In [ ]:
housing_num = housing.drop("ocean_proximity", axis=1)

In [ ]:
housing.ocean_proximity.value_counts()

In [ ]:
# вспоминаем про наше желание добавить новые атрибуты
from sklearn.base import BaseEstimator, TransformerMixin

# column index
rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]


In [ ]:
# pipeline для обработки числовых данных
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, Imputer

num_pipeline = Pipeline([
    ('imputer', Imputer(strategy="median")),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler()),
])

In [ ]:
from sklearn.preprocessing import LabelBinarizer

class PipelineFriendlyLabelBinarizer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None,**fit_params):
        return self
    def transform(self, X):
        return LabelBinarizer().fit(X).transform(X)

# pipeline для обработки категорийных данных
cat_pipeline = Pipeline([
    ('label_binarizer', PipelineFriendlyLabelBinarizer()),
])

In [ ]:
# мы хотим объединить их, для этого нужен селектор для DF (чтобы выбирать нужные поля для нужного пайплайна)
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [ ]:
num_attribs = housing.columns.drop("ocean_proximity")
cat_attribs = ["ocean_proximity"]

num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_attribs)),
    ('imputer', Imputer(strategy="median")),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler()),
])

cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat_attribs)),
    ('label_binarizer', PipelineFriendlyLabelBinarizer()),
])



In [ ]:
from sklearn.pipeline import FeatureUnion

full_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline", num_pipeline),
    ("cat_pipeline", cat_pipeline),
])

In [ ]:
housing_prepared = full_pipeline.fit_transform(housing)
housing_prepared

In [ ]:
housing_prepared.shape

# Build model

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

In [ ]:
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
# Training error
lin_rmse

In [ ]:
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)
final_predictions = lin_reg.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse